In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [30]:
import nltk
from nltk.corpus import gutenberg

# Get the raw text of "Emma" by Jane Austen
text = gutenberg.raw("austen-emma.txt")

# Print the first 500 characters of the text
print(text[:500])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t


In [31]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '6', '7', '8', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [32]:
vocab_size

77

In [33]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [34]:
x = 'Hello World!'

In [35]:
encoded = encode(x)

In [36]:
encoded

[29, 55, 62, 62, 65, 1, 44, 65, 68, 62, 54, 2]

In [37]:
decode(encoded)

'Hello World!'

In [38]:
data = torch.tensor(encode(text), dtype=torch.long)

In [39]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[59, 70, 55,  1, 56, 59, 68, 63],
        [55,  1, 51,  1, 66, 65, 59, 64],
        [ 8,  1, 62, 65, 65, 61, 59, 64],
        [ 1, 56, 68, 65, 63,  1, 70, 58]], device='cuda:0')
targets:
tensor([[70, 55,  1, 56, 59, 68, 63, 10],
        [ 1, 51,  1, 66, 65, 59, 64, 70],
        [ 1, 62, 65, 65, 61, 59, 64, 57],
        [56, 68, 65, 63,  1, 70, 58, 55]], device='cuda:0')


In [40]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


2DpI(K!BooYr0pL;;ppVf&c3kY:vd`2uMXhy4_JHgYfpJhNN] i]Yr'u'DFE)"28g&y60[GaGoaeTt0MejqJ4jR.A2vlIhvkJsw]0Il0f_q3IV&vV F]`!L kUl[UsbezARz!yt:siFxPg
Js?saH,pIFI
)st 6bUm7jvwozakw.wYDqUFTizIg&GnjeSG_xdH-Ov,ifBvCvmP!:DhjN7Wg&7gSAnjmfQaDQG8"tF
hh[2aNfsaCEi0Oy_]yBH_[f_lb3OR&pV;P7HrKu2!vuTC)J:sbjcJ?`]y VXEvjeB1ARi1xj&P 6fw6GnwoF26iKs2nw;PE6_DHT4tG3)IEoaig
m0)sCPytNk;.&CdSMK!:
b'R"iwMIoPyL `OR0'?ft3X]Hj"F_l_ntqJRO"xdoYpL`NgDz2Xzob7hUf`&HT4rs('
Bcd77Oi_.:Gx8pppL k2n4jH2xFQKsaBM.7aY1D2b["2Jvo_TY?Cd`y[RHqDwUas


In [41]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [42]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.714, val loss: 4.718
step: 250, train loss: 4.655, val loss: 4.651
step: 500, train loss: 4.574, val loss: 4.595
step: 750, train loss: 4.503, val loss: 4.529
4.6142191886901855


In [43]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


oRVTV_PIVa[EuhMIwocTimxyzgM,;KUja.o;pB&0e6[v,zvPu[_xQql0vLGj(,3gA68I1DAj(giaGk'_ehzgy_p.D[fU4jLGt!n])pv]yOr3_X]T)O:wJaG8oBotoBohoFy4xztf;0Jg
JyPo_3gIhG82RvU.pBpB[S?-Iw.
'K
cDQGKtvlWdI?.bgm0?LB;hLlI;j"iz8y_Dh B)D.Anwn]]GtM_a3GG[cHDpaqV&GkvDSM.H)e 8A`]lFsw0k),z7hWejo(p0O_jhD82u
2L F;.oa"?HwC;)HDPb'Dkju;q&
QL3)6_,as2"hXM("D2cu62atE0slFp(zU)G]sIcWVx[q1YS'lIUiw82T:wHgoDmF-PJ37G`qe&8LaTKSHJ.7WW(yQ[VDi!Uu)?_U8vl0stHcDAckJwny-Aaq1t"oPocuomAn.3g&.
 ?J"gV
dkihCqXpwo)RW:wAC7&-]imfbbVIDmjeiccFka.jkv[vvC!?cW
